In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Activation, ZeroPadding2D
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.optimizers import Adadelta

num_classes = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [4]:
model = Sequential()

model.add(Conv2D(96, (11,11), input_shape=x_train.shape[1:], padding="same", kernel_regularizer=l2(0)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(256, (5,5), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, (3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, (3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(rate=0.5))

model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(rate=0.5))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation("softmax"))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 96)        384       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 256)      

In [5]:
batch_size=16
epochs = 4

model.compile(loss="categorical_crossentropy", optimizer=Adadelta(), metrics=["accuracy"])
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(x_test,y_test), shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/4
50000/50000 [==============================] - 362s 7ms/step - loss: 1.5245 - accuracy: 0.4567 - val_loss: 1.3695 - val_accuracy: 0.5280
Epoch 2/4
50000/50000 [==============================] - 351s 7ms/step - loss: 1.1509 - accuracy: 0.6028 - val_loss: 1.1154 - val_accuracy: 0.6115
Epoch 3/4
50000/50000 [==============================] - 353s 7ms/step - loss: 0.9497 - accuracy: 0.6794 - val_loss: 1.0184 - val_accuracy: 0.6483
Epoch 4/4
50000/50000 [==============================] - 355s 7ms/step - loss: 0.8015 - accuracy: 0.7300 - val_loss: 0.8677 - val_accuracy: 0.7065


In [6]:
score = model.evaluate(x_test, y_test)

print("Test loss ", score[0])
print("Test acc ", score[1] )

10000/10000 [==============================] - 7s 719us/step
Test loss  0.867741745185852
Test acc  0.7064999938011169
